In [14]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.optimize import curve_fit
from pandas import DataFrame as df
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import get_decimal_separator
from Data_setup_FirstPaper import data_set
import os
from galvani import BioLogic
currentcorrect = 1000 #converts to mA to A in the log
areacorrect = 1 #converts the surface area in the log
#start figure, assign colors 
fig = go.Figure()
fig.update_xaxes(range=[-6, -1.0],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[-0.2, 1.2],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
colors = px.colors.qualitative.Safe

fig.update_layout(
    xaxis_title="Log Current (mA/cm<sup>2</sup>)",
    yaxis_title="Overpotential, η(V) - iR<sub>u</sub>",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey",
    ),
    width = 500,
    height = 500,
)
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(0, 0, 0, 0)', #sets the background to be transparent... might appear black in .jpeg but will be transparent in the .svg
})
fig.update_layout(
    )
#load in data either .mpr or .mpt
for i in data_set:
    
    file = data_set[i]['data_Tafel']
    if not os.path.exists(file):
        continue
    if file.endswith('mpt'):
        headerlength = get_header_length(file)
        data = pd.read_csv(file, encoding='ansi', sep="\t", decimal=get_decimal_separator(file),  skiprows=range(headerlength))
        current_column = '<I>/mA'
    else:
        mpr_file = BioLogic.MPRfile(file)
        data = pd.DataFrame(mpr_file.data)
        current_column = 'I/mA'
    if not os.path.exists(data_set[i]['data_Tafel']):
        continue
    
    #define variables, sets up number of experiments, does the voltage correction based on Ru and 
    exp = data['Ns']
    Nexp = exp[-1:] + 1
    voltage = data['Ewe/V']
    current = data['control/mA']
    current = current/data_set[i]['mass']
    voltage_corrected = data['Ewe/V'] - data['control/mA'] * data_set[i]['Ru']/1000 - data_set[i]['cal'] #does the iR correction converts to overpotential
    #creates numpy array Tafel with the voltage, voltage error, and logJ from the data 
    j = 0
    Tafel = []
    for j in range(int(Nexp)):
        v = df.mean(voltage_corrected.loc[exp==j])
        verr = df.std(voltage_corrected.loc[exp==j])
        J = current / currentcorrect / areacorrect
        logJ = np.log10(J.loc[exp==j])
        logJ = logJ.iloc[0:1] 
        logJ = float(logJ)
        logJ = np.round(logJ,2)
        Tafel.append([v, verr, logJ])   
    Tafel = np.array(Tafel)
    
    #calculate fits 
    
    x = np.array(Tafel[:,2])
    y = np.array(Tafel[:,0])
    fit_min = -6
    fit_max = -0
    mask = (x >= fit_min) & (x <= fit_max)
    x_fit = x[mask]
    y_fit = y[mask]
    z, cov = np.polyfit(x_fit, y_fit, 1, cov=True)
    slope = z[0]
    intercept = z[1]
    slope_error = np.sqrt(cov[0,0])
    intercept_error = np.sqrt(cov[1,1])
    print(f"{data_set[i]['label']}: slope={slope*1000:.0f}±{slope_error*1000:.0f} mV/decade, Y-intercept={intercept:.2f}±{intercept_error:.2f}")
    
    x_pred = np.linspace(fit_min, fit_max, num=50)
    y_pred = np.polyval(z, x_pred)
    fig.add_trace(go.Scatter(
        x=x_pred,
        y=y_pred,
        mode='lines',
        showlegend= False,
        line_color=colors[data_set[i]['color_index']]
))

    fig.add_trace(go.Scatter(
        x=Tafel[:,2],
        y=Tafel[:,0],
        error_y=dict(
            type='data',
            
            array=Tafel[:,1],
            visible=True),
        mode='markers',
        name= data_set[i]['label'],
        line_color = colors[data_set[i]['color_index']],
    ),
    
   
)


## convert your array into a dataframe

fig.show()
fig.write_image('Tafel_firstpaperall.jpeg')
fig.write_image('Tafel.firstpaperall.svg')


IrO<sub>x</sub>: slope=208±23 mV/decade, Y-intercept=1.01±0.09
Ni<sub>60</sub>Co<sub>30</sub>Fe<sub>10</sub>O<sub>x</sub>: slope=330±25 mV/decade, Y-intercept=1.78±0.09
FeN<sub>x</sub>: slope=192±13 mV/decade, Y-intercept=1.20±0.05
CoN<sub>x</sub>: slope=313±17 mV/decade, Y-intercept=1.57±0.06
NiN<sub>x</sub>: slope=341±29 mV/decade, Y-intercept=1.99±0.11
Co<sub>30</sub>Ni<sub>60</sub>Fe<sub>10</sub>N<sub>x</sub>: slope=210±9 mV/decade, Y-intercept=1.01±0.03
Co<sub>45</sub>Ni<sub>45</sub>Fe<sub>10</sub>N<sub>x</sub>: slope=175±12 mV/decade, Y-intercept=0.86±0.04
Co<sub>57</sub>Ni<sub>14</sub>Fe<sub>29</sub>N<sub>x</sub>: slope=64±5 mV/decade, Y-intercept=0.51±0.02
